In [58]:
import sys
import requests
import numpy as np
import pandas as pd 
import json

sys.path.append("../")
from src.fwi_system_calculator import FWISystemCalculator

In [41]:
def get_inca_data(parameter: list = "T2M",
                  start_date: str = '2021-08-01T12:00',
                  end_date: str = '2021-08-02T12:00',
                  bbox: list = [47.45, 14.05, 47.50, 14.10],
                  output_format: str = "geojson") -> dict:

    base_url = "https://dataset.api.hub.geosphere.at/v1/grid/historical/inca-v1-1h-1km"
    
    # Constructing the URL with parameters
    url = f"{base_url}?parameters={parameter}&start={start_date}&end={end_date}&bbox={','.join(map(str, bbox))}&output_format={output_format}"

    try:
        # Making the HTTP request
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad responses (4xx and 5xx)
        
        # Returning the data as text
        return json.loads(response.text)
    
    except requests.exceptions.RequestException as e:
        # Handling any exceptions that may occur during the request
        print(f"Error: {e}")
        return None
    
def extract_inca_data(data: dict, parameter_name: str) -> pd.DataFrame:
    data_prep = {parameter_name: [], "lon": [], "lat": []}
    for row in data["features"]:
        lon = row["geometry"]["coordinates"][0]
        lat = row["geometry"]["coordinates"][1]
        if parameter_name == "RR":
            data_agg = np.sum(row["properties"]["parameters"][parameter_name]["data"])
        else:
            data_agg = row["properties"]["parameters"][parameter_name]["data"][0]
        data_prep["lon"].append(lon)
        data_prep["lat"].append(lat)
        data_prep[parameter_name].append(data_agg)
    return pd.DataFrame(data_prep)

def calculate_wind_speed(uu: float, vv: float):
    return np.sqrt(uu**2 + vv**2)


In [42]:
t2m_data = get_inca_data(parameter="T2M", start_date='2021-08-02T12:00', end_date='2021-08-02T12:00', bbox=[47.45, 14.05, 47.50, 14.10], output_format="geojson")
rr_data = get_inca_data(parameter="RR", start_date='2021-08-01T12:00', end_date='2021-08-02T12:00', bbox=[47.45, 14.05, 47.50, 14.10], output_format="geojson")
rhum_data = get_inca_data(parameter="RH2M", start_date='2021-08-02T12:00', end_date='2021-08-02T12:00', bbox=[47.45, 14.05, 47.50, 14.10], output_format="geojson")
uu_data = get_inca_data(parameter="UU", start_date='2021-08-01T12:00', end_date='2021-08-02T12:00', bbox=[47.45, 14.05, 47.50, 14.10], output_format="geojson")
vv_data = get_inca_data(parameter="VV", start_date='2021-08-01T12:00', end_date='2021-08-02T12:00', bbox=[47.45, 14.05, 47.50, 14.10], output_format="geojson")

In [51]:
t2m_data_extracted = extract_inca_data(t2m_data, "T2M").loc[:, ["T2M"]]
rr_data_extracted = extract_inca_data(rr_data, "RR").loc[:, ["RR"]]
rhum_data_extracted = extract_inca_data(rhum_data, "RH2M").loc[:, ["RH2M"]]
uu_data_extracted = extract_inca_data(uu_data, "UU").loc[:, ["UU"]]
vv_data_extracted = extract_inca_data(vv_data, "VV")

dfs_to_merge = [t2m_data_extracted, rr_data_extracted, rhum_data_extracted, uu_data_extracted, vv_data_extracted]
df = pd.concat(dfs_to_merge, axis=1)

In [55]:
# calculate wind speed
df["wind_speed"] = df.apply(lambda x: calculate_wind_speed(x.UU, x.VV), axis=1)
df.head()

,T2M,RR,RH2M,UU,VV,lon,lat,wind_speed
0,15.71,13.833,56.28,0.55,-1.76,14.062182,47.452194,1.843936
1,15.44,13.874,57.31,3.93,0.20,14.062306,47.461189,3.935086
2,12.88,14.283,70.69,4.98,-0.12,14.062430,47.470188,4.981446
3,14.18,13.882,68.66,4.29,-0.72,14.062554,47.479183,4.350000
4,18.37,13.624,50.95,1.48,-0.08,14.062678,47.488182,1.482161


In [60]:
# calculate FFMC
def calculate_fwi(row):
    calculator = FWISystemCalculator(row['T2M'], row['RH2M'], row['wind_speed'], row['RR'])
    ffmc = calculator.ffmc_calc(85)  # assuming ffmc0 is defined somewhere
    return pd.Series({'FFMC': ffmc})

# Assume df is your DataFrame with columns 'temp', 'rhum', 'wind', 'prcp'
# Also, assume ffmc0, dmc0, dc0, and mth are defined somewhere
df_result = df.apply(calculate_fwi, axis=1)

# Concatenate the original DataFrame with the result DataFrame
result_df = pd.concat([df, df_result], axis=1)

In [61]:
result_df.head()

,T2M,RR,RH2M,UU,VV,lon,lat,wind_speed,FFMC
0,15.71,13.833,56.28,0.55,-1.76,14.062182,47.452194,1.843936,44.476337
1,15.44,13.874,57.31,3.93,0.20,14.062306,47.461189,3.935086,46.715802
2,12.88,14.283,70.69,4.98,-0.12,14.062430,47.470188,4.981446,39.454863
3,14.18,13.882,68.66,4.29,-0.72,14.062554,47.479183,4.350000,41.101060
4,18.37,13.624,50.95,1.48,-0.08,14.062678,47.488182,1.482161,48.691021


In [ ]:
# TODO write to script (delete notebook)
# TODO save as csv
